In [1]:
using ReactiveMP, Rocket, GraphPPL, Random, LinearAlgebra, Plots, Flux, Zygote, ForwardDiff, DataFrames, DelimitedFiles, Plots, StableRNGs

In [20]:
function generate_based_on_bernoulli(bns)
    if bns == 0
        return 10
    end
    return 0
end
@model function control_flow(n, n_iterations, n_samples, rng, learning_rate)

    # z = randomvar()
    x = randomvar()
    y = datavar(Float64, n)

    p ~ Beta(1, 1)
    b ~ Bernoulli(p)
    x ~ generate_based_on_bernoulli(b) where {meta = CVIApproximation(n_iterations, n_samples, rng, Descent(learning_rate))}
    
    for t in 1:n
        y[t] ~ NormalMeanVariance(x, 1)
    end

    return b, p, x, y
end

constraints = @constraints begin
    q(x, b, p) = q(b)q(x)q(p)
end;

In [21]:
function generate_sample(gold_p, n)
    x1 = rand(NormalMeanVariance(10, 1), n)
    x2 = rand(NormalMeanVariance(0, 1), n)
    selector = rand(Bernoulli(gold_p), n)
    return x1 .* selector + (1 .- selector) .* x2
end

sample = generate_sample(0.6, 1000);

In [26]:
res = inference(
    model = Model(control_flow, length(sample), 1000, 2000, nothing, 0.01),
    data = (y = sample,),
    iterations = 2,
    free_energy = false,
    returnvars = (μ = KeepEach()),
    constraints = constraints,
    initmessages = (p = Beta(1, 1),),
    initmarginals = (p = Beta(1, 1),),
    showprogress = false,
)

MethodError: MethodError: Cannot `convert` an object of type Vector{Float64} to an object of type Float64
Closest candidates are:
  convert(::Type{T}, !Matched::Gray24) where T<:Real at ~/.julia/packages/ColorTypes/1dGw6/src/conversions.jl:114
  convert(::Type{T}, !Matched::Gray) where T<:Real at ~/.julia/packages/ColorTypes/1dGw6/src/conversions.jl:113
  convert(::Type{T}, !Matched::Base.TwicePrecision) where T<:Number at twiceprecision.jl:273
  ...